In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
# import logging
# import warnings

# logging.basicConfig(level=logging.DEBUG)
# warnings.filterwarnings('ignore')

In [4]:
import tensorflow as tf

tf.compat.v1.enable_eager_execution()

In [5]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import glowtts
import tensorflow as tf
import numpy as np
import math






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [6]:
config = malaya_speech.config.fastspeech2_config
config['encoder_hidden_size'] = 192
config['encoder_num_hidden_layers'] = 6
config['encoder_attention_head_size'] = 32
config['encoder_intermediate_size'] = 768
config = glowtts.Config(vocab_size = 66, **config)
config_glowtts = glowtts.Config_GlowTTS(malaya_speech.config.glowtts_config)

In [7]:
i = tf.placeholder(tf.int32, [None, None])
i_lengths = tf.placeholder(tf.int32, [None])
mel_outputs = tf.placeholder(tf.float32, [None, None, 80])
mel_lengths = tf.placeholder(tf.int32, [None])

RuntimeError: tf.placeholder() is not compatible with eager execution.

In [8]:
import pickle

with open('dataset-mel.pkl', 'rb') as fopen:
    data, d = pickle.load(fopen)
    
with open('dataset-mel-wav.pkl', 'rb') as fopen:
    wav = pickle.load(fopen)
    
data.keys()

dict_keys(['mel', 'text_ids', 'len_mel', 'len_text_ids', 'stop_token_target', 'f0', 'len_f0', 'energy', 'len_energy', 'g'])

In [9]:
data['text_ids'][0].shape

(72,)

In [10]:
i = tf.convert_to_tensor([data['text_ids'][0],data['text_ids'][0]])
i_lengths = tf.convert_to_tensor([data['len_text_ids'][0,0], data['len_text_ids'][0,0]])
mel_outputs = tf.convert_to_tensor([data['mel'].astype(np.float32)[0],data['mel'].astype(np.float32)[0]])
mel_lengths = tf.convert_to_tensor([408,408])

In [11]:
model = glowtts.Model(config, config_glowtts)

In [12]:
(z, z_m, z_logs, logdet, z_mask), (x_m, x_logs, x_mask), (attn, logw, logw_) = model(i, y = mel_outputs, y_lengths = mel_lengths, training = True)
(z, z_m, z_logs, logdet, z_mask), (x_m, x_logs, x_mask), (attn, logw, logw_)

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


((<tf.Tensor: id=14580, shape=(2, 408, 80), dtype=float32, numpy=
  array([[[-7.7444324 ,  1.763787  , -8.422806  , ...,  0.67502844,
           -4.355525  ,  0.9894304 ],
          [-4.193779  ,  1.8496653 , -5.171345  , ...,  0.99112004,
           -2.5143926 ,  0.98881185],
          [-7.1253967 ,  1.756922  , -7.2872715 , ...,  0.7170011 ,
           -3.7818403 ,  0.76780576],
          ...,
          [-4.3675623 ,  2.0695858 , -4.3675623 , ...,  2.0695858 ,
           -4.3675623 ,  2.0695858 ],
          [-7.7083645 ,  1.432734  , -7.7083645 , ...,  1.432734  ,
           -7.7083645 ,  1.432734  ],
          [-4.3675623 ,  2.0695858 , -4.3675623 , ...,  2.0695858 ,
           -4.3675623 ,  2.0695858 ]],
  
         [[-7.7444324 ,  1.763787  , -8.422806  , ...,  0.67502844,
           -4.355525  ,  0.9894304 ],
          [-4.193779  ,  1.8496653 , -5.171345  , ...,  0.99112004,
           -2.5143926 ,  0.98881185],
          [-7.1253967 ,  1.756922  , -7.2872715 , ...,  0.7170011 ,

In [15]:
x_logs

<tf.Tensor: id=2817, shape=(2, 72, 80), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)>

In [17]:
z.shape

TensorShape([Dimension(2), Dimension(408), Dimension(80)])

In [30]:
attn

<tf.Tensor: id=14623, shape=(2, 1, 72, 408), dtype=float32, numpy=
array([[[[1., 0., 0., ..., 0., 0., 0.],
         [0., 1., 1., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]],


       [[[1., 0., 0., ..., 0., 0., 0.],
         [0., 1., 1., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]]], dtype=float32)>

In [20]:
# commons.mle_loss(z, z_m, z_logs, logdet, z_mask)

def mle_loss(z, m, logs, logdet, mask):
    l = tf.reduce_sum(logs) + 0.5 * tf.reduce_sum(tf.math.exp(-2 * logs) * ((z - m)**2))
    l = l - tf.reduce_sum(logdet)
    l = l / tf.reduce_sum(tf.ones_like(z) * mask)
    l = l + 0.5 * math.log(2 * math.pi)
    return l

l_mle = mle_loss(z, z_m, z_logs, logdet, z_mask)
l_mle

<tf.Tensor: id=14662, shape=(), dtype=float32, numpy=2.0551758>

In [24]:
z_mask

<tf.Tensor: id=3057, shape=(2, 408, 1), dtype=float32, numpy=
array([[[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],


In [21]:
mle_loss(z, z, z_logs, logdet, z_mask)

<tf.Tensor: id=14688, shape=(), dtype=float32, numpy=0.9189385>

In [ ]:
# commons.duration_loss(logw, logw_, x_lengths)

def duration_loss(logw, logw_, lengths):
    # l = torch.sum((logw - logw_)**2) / torch.sum(lengths)
    l = tf.reduce_sum((logw - logw_)**2) / tf.reduce_sum(tf.cast(lengths, tf.float32))
    return l

l_length = duration_loss(logw, logw_, i_lengths)
l_length

In [ ]:
loss_g = l_mle + l_length

In [ ]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate = 1e-4).minimize(loss_g)

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
o_ = sess.run([l_mle, l_length], feed_dict = {i: data['text_ids'],
                                   i_lengths: data['len_text_ids'][:,0],
                        mel_outputs: data['mel'],
                        mel_lengths: data['len_mel'][:,0]})
o_

In [ ]:
o_ = sess.run([attn, logw, logw_, x_mask], feed_dict = {i: data['text_ids'],
                                   i_lengths: data['len_text_ids'][:,0],
                        mel_outputs: data['mel'],
                        mel_lengths: data['len_mel'][:,0]})
o_

In [ ]:
sess.run(model.decoder.flows[0].bias)

In [ ]:
model.decoder.flows[2].start.remove()

In [ ]:
for k in range(300):
    print(k, sess.run([optimizer, l_mle, l_length], feed_dict = {i: data['text_ids'],
                                       i_lengths: data['len_text_ids'][:,0],
                            mel_outputs: data['mel'],
                            mel_lengths: data['len_mel'][:,0]}))

In [ ]:
tf.trainable_variables()

In [ ]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'test/model.ckpt')

In [ ]:
# !ls -lh test
# !rm -rf test